<h1 align=center><font size="6"> 
Genome Sequences</font></h1>
<h2 align=center><font size="3"> Author: NhanTV</font></h2>

One will simulate the process of genome sequencing simply without considering which alleles and chromosomes gens belong to.

<h1>Table of contents</h1>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <ol>
        <li><a href="#data">Data</a></li>
            <ol>
                <li><a href="#import dataset">Import Dataset</a></li>
                <li><a href="#getting compisition">Getting Composition (Paired Reads)</a></li>
            </ol>
        <li><a href="#contructing the de bruijn">Constructing the De Bruijn Graph</a></li>          
        <li><a href="#eulerian path">Eulerian Path</a></li>
        <li><a href="#non branching paths">Maximal Non-Branching Paths in the De Bruijn Graph</a></li>
    </ol>
</div>
<br>
<hr>

<h1 id = "data">1. Data</h1>
Genome Data of <b>Ecoli bacterium</b> can be downloaded from here: <a href="https://github.com/nhanta/Genome-Sequences/blob/master/ecoli.txt">Ecoli.txt</a>

In [456]:
import numpy as np
import pandas as pd

<h3 id = "import dataset"> Import Dataset</h3>

For easy considering of the sequencing process on a personal computer, we choose the first 100 nucleotides.

In [457]:
with open ("D:/Data Science/Coursera/Bioinformatics/Paired reads/ecoli.txt", "r") as file:
    text = file.read().replace('\n', '')
    print('The length of genome:', len(text))
    text = text[:100]
    print('Genome Sequences:', text)

The length of genome: 4639675
Genome Sequences: AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAAT


<h3 id = "getting composition">Getting Composition (Paired Reads)</h3>

In [458]:
def get_composition(text, k):
    pattern = []
    for i in range(len(text)-k +1):
        pattern.append(text[i:i+k])
    return(pattern)

We will to composite the text into (k, d)-mers so that it's surfix: (k-1, d)-mers equals to the prefix (k-1, d)-mers of the next k-mers.

In [459]:
def get_composition_d(text, k, d):
    pattern = []
    for i in range(len(text)-2*k-d+1):
        pattern.append(text[i:i+k]+'|'+ text[i+k+d:i+2*k+d])
    return(pattern)

In [460]:
pre_paired_reads = get_composition_d(text, 4, 2)
pre_paired_reads[:5]

['AGCT|TCAT', 'GCTT|CATT', 'CTTT|ATTC', 'TTTT|TTCT', 'TTTC|TCTG']

In [461]:
paired_reads= sorted(pre_paired_reads)
paired_reads[:5]

['AAAA|AGAG', 'AAAA|AGTG', 'AAAA|GAGT', 'AAAA|GTGT', 'AAAG|TGTC']

<h1 id = "contructing the de bruijn">2. Contructing the De Bruijn Graph</h1>

In [463]:
def paired_vertexes(text, k):
    t_pair = []
    for item in text:
        prefix = get_composition(item[:k], k-1)[0] + '|' + get_composition(item[k+1:], k-1)[0]
        surfix = get_composition(item[:k], k-1)[1] + '|' + get_composition(item[k+1:], k-1)[1]
        t_pair.append([prefix, surfix])
    return(t_pair)

In [464]:
paired_ver = paired_vertexes(paired_reads, 4)
paired_ver[:5]

[['AAA|AGA', 'AAA|GAG'],
 ['AAA|AGT', 'AAA|GTG'],
 ['AAA|GAG', 'AAA|AGT'],
 ['AAA|GTG', 'AAA|TGT'],
 ['AAA|TGT', 'AAG|GTC']]

In [465]:
def get_DeBruijn_k_d_mer(t):
    count = []
    for i in range(len(t)): 
        h = [t[i][1]]
        for j in range(len(t)):
            if j > i and t[i][0] == t[j][0]:
                h.append(t[j][1])
                count.append(j)
                
        t[i][1] = h
        
    count = np.unique(count).tolist()

    for c in count[::-1]:
        t.remove(t[c])
    return(t)

In [466]:
graph = get_DeBruijn_k_d_mer(paired_ver)
graph[:5]

[['AAA|AGA', ['AAA|GAG']],
 ['AAA|AGT', ['AAA|GTG']],
 ['AAA|GAG', ['AAA|AGT']],
 ['AAA|GTG', ['AAA|TGT']],
 ['AAA|TGT', ['AAG|GTC']]]

<h1 id = "eulerian path">3. Eulerian Path</h1>

<h3> Getting Vertexes from de Bruijn Graph</h3> 

In [467]:
def ver(graph):
    ver = []
    for edge in graph:
        ver.append(edge[0])
        ver.append(edge[1])
    ver = np.unique(ver)
    return (ver)

<h3>Definition of Eulerian Path</h3>

In [468]:
def get_eulerianPath(graph):
        graph = [(src,dst) for src,dst in graph]
        currentVertex = verifyAndGetStart(graph)
        path = [currentVertex]
        # "next" is where vertices get inserted into our tour
        # it starts at the end (i.e. it is the same as appending),
        # but later "side-trips" will insert in the middle
        next = 1
        while len(graph) > 0:
            # follows a path until it ends
            for edge in graph:
                if (edge[0] == currentVertex):
                    currentVertex = edge[1]
                    graph.remove(edge)
                    path.insert(next, currentVertex)
                    next += 1
                    break
            else:
                # Look for side-trips along the path
                for edge in graph:
                    try:
                        # insert our side-trip after the
                        # "u" vertex that is starts from
                        next = path.index(edge[0]) + 1
                        currentVertex = edge[0]
                        break
                    except ValueError:
                        continue
                else:
                    print ("There is no path!")
                    return False
        return path
    
# More new methods for the Graph Class
def degrees(graph):
        """ Returns two dictionaries with the inDegree and outDegree
        of each node from the graph. """
        inDegree = {}
        outDegree = {}
        for src, dst in graph:
            outDegree[src] = outDegree.get(src, 0) + 1
            inDegree[dst] = inDegree.get(dst, 0) + 1
        return (inDegree, outDegree)
            
def verifyAndGetStart(graph):
        inDegree, outDegree = degrees(graph)
        start = 0
        end = 0
        vertex = ver(graph)
        i = []
        o = []
        for vert in vertex:
            i.append(inDegree.get(vert,0))
            o.append(outDegree.get(vert,0))
            
        if (np.array(i) - np.array(o)).any() != np.zeros((len(i)), dtype=int).any():
            for vert in vertex:
                ins = inDegree.get(vert,0)
                outs = outDegree.get(vert,0)
                if (ins == outs):
                    continue
                elif (ins - outs == 1):
                    end = vert
                elif (outs - ins == 1):
                    start = vert
                else:
                    start, end = 'no', 'no'
                    break
            if (start != 'no') and (end != 'no'):
                return (start)
            else:
                return('no')
        else:
            return (vertex[0])
        

<h1 id = "non branching paths">4. Maximal Non-Branching Paths in the De Bruijn Graph</h1>

For some reason, the result of the gene sequencer lacks some nucleotides. This causes the De Bruijn graph to lack some edges, and sequenced gene is incorect. Therefore, we need to divide the De Bruijn graph into <b>Maximal Non-Branching Paths</b> for sequencing instead of sequencing the whole result.

In [469]:
def change_graph(graph):
    ch_graph = []
    for x in graph:
        l = len(x[1])
        for i in range(l):
            if x[1][i] != ',':
                ch_graph.append((x[0], x[1][i]))
    return(ch_graph)

In [470]:
debruijn_graph = change_graph(graph)
debruijn_graph[:5]

[('AAA|AGA', 'AAA|GAG'),
 ('AAA|AGT', 'AAA|GTG'),
 ('AAA|GAG', 'AAA|AGT'),
 ('AAA|GTG', 'AAA|TGT'),
 ('AAA|TGT', 'AAG|GTC')]

In [471]:
def get_edge_from_v (v, Graph):
    ed = []
    for edge in Graph:
        if v == edge[0]:
            ed.append((v, edge[1]))
    return (ed)
            
def MaximalNonBranchingPaths(Graph):
    inDegree, outDegree = degrees(Graph)
    Paths = []
    vertex = ver(Graph)
    new_graph = Graph[:]
    
    for v in vertex:
        ins = inDegree.get(v,0)
        outs = outDegree.get(v,0)
        if ins != 1 or outs != 1:
            if outs > 0:
                for edge in get_edge_from_v (v, new_graph): 
                    NonBrachingPath = [edge]
                    w = edge[1]
                    new_graph.remove(edge)
                    
                    while inDegree.get(w, 0) == 1 and outDegree.get(w, 0) == 1:
                        for w_edge in get_edge_from_v(w, new_graph):
                            NonBrachingPath.append(w_edge)
                            w = w_edge[1]
                            new_graph.remove(w_edge)
                    Paths.append(NonBrachingPath) 
                    
        elif ins == 1 and outs == 1:
            first_remain = get_edge_from_v(v, new_graph)
            
            for edge in first_remain:
                NonBranchingPath = [edge]
                w = edge[1]
                ins_2 = inDegree.get(w, 0)
                outs_2 = outDegree.get(w, 0)
                
                while outs_2 == 1:
                    
                    remain_edge = get_edge_from_v(w, new_graph)
                    
                    for w_edge in remain_edge:
                        
                        if w_edge != edge:
                            
                            NonBranchingPath.append(w_edge)
                            w = w_edge[1]
                   
                    if NonBranchingPath[0][0] == NonBranchingPath[-1][-1]:
                        Paths.append(NonBranchingPath)
                        for i in NonBranchingPath:
                            new_graph.remove(i)  
                        break
                        
                    elif NonBranchingPath[0][0] != NonBranchingPath[-1][-1] and remain_edge == []:    
                        break
            
    showing_path = []
    for p in Paths:
        g = get_eulerianPath(p)
        path = '->'.join(g)
        showing_path.append(path)
    
    return(showing_path)

In [483]:
Max_NonBanching_Paths = MaximalNonBranchingPaths(debruijn_graph)
Max_NonBanching_Paths

['AGC|TCA->GCT|CAT->CTT|ATT->TTT|TTC->TTT|TCT->TTC|CTG',
 'AAA|AGA->AAA|GAG->AAA|AGT->AAA|GTG->AAA|TGT->AAG|GTC->AGA|TCT->GAG|CTG->AGT|TGA->GTG|GAT->TGT|ATA->GTC|TAG->TCT|AGC->CTG|GCA->TGA|CAA->GAC|AAC->ACT|ACG->CTG|CGG->TGC|GGG->GCA|GGC->CAA|GCA->AAC|CAA->ACG|AAT->CGG|ATA->GGG|TAT->GGC|ATG->GCA|TGT->CAA|GTC->AAT|TCT->ATA|CTC->TAT|TCT->ATG|CTG->TGT|TGT->GTC|GTG->TCT|TGT->CTC|GTG->TCT|TGG->CTG|GGA->TGT|GAT->GTG|ATT->TGT|TTA->GTG|TAA->TGG|AAA->GGA|AAA->GAT|AAA->ATT|AAA->TTA|AAA->TAA|AAG->AAA|AGA',
 'CTG|GCA->TGA|CAG->GAT|AGC->ATA|GCT->TAG|CTT->AGC|TTC->GCA|TCT->CAG|CTG->AGC|TGA->GCT|GAA->CTT|AAC->TTC|ACT->TCT|CTG->CTG|TGG->TGA|GGT->GAA|GTT->AAC|TTA->ACT|TAC->CTG|ACC->TGG|CCT->GGT|CTG->GTT|TGC->TTA|GCC->TAC|CCG->ACC|CGT->CCT|GTG->CTG|TGA->TGC|GAG->GCC|AGT->CCG|GTA->CGT|TAA->GTG|AAA->TGA|AAT',
 'ATT|ACT->TTC|CTG->TCA|TGA->CAT|GAC->ATT|ACT',
 'TTC|CTG->TCT|TGC->CTG|GCA']

In [484]:
print('Total number of contigs is:',len(Max_NonBanching_Paths))

Total number of contigs is: 5


Suppose we choose any contig to get a genome branch.

In [485]:
Branch = Max_NonBanching_Paths[1]
Branch

'AAA|AGA->AAA|GAG->AAA|AGT->AAA|GTG->AAA|TGT->AAG|GTC->AGA|TCT->GAG|CTG->AGT|TGA->GTG|GAT->TGT|ATA->GTC|TAG->TCT|AGC->CTG|GCA->TGA|CAA->GAC|AAC->ACT|ACG->CTG|CGG->TGC|GGG->GCA|GGC->CAA|GCA->AAC|CAA->ACG|AAT->CGG|ATA->GGG|TAT->GGC|ATG->GCA|TGT->CAA|GTC->AAT|TCT->ATA|CTC->TAT|TCT->ATG|CTG->TGT|TGT->GTC|GTG->TCT|TGT->CTC|GTG->TCT|TGG->CTG|GGA->TGT|GAT->GTG|ATT->TGT|TTA->GTG|TAA->TGG|AAA->GGA|AAA->GAT|AAA->ATT|AAA->TTA|AAA->TAA|AAG->AAA|AGA'

In [486]:
vertexes = Branch.split('->')
vertexes[:5]

['AAA|AGA', 'AAA|GAG', 'AAA|AGT', 'AAA|GTG', 'AAA|TGT']

In [487]:
def get_genome_string(text, k):
    st = str()
    it = len(text)
    for i in range(it):
        st = st + text[i][0]
    if k > 1:
        st = st + text[it-1][-k+1:-1] + text[it-1][-1]
    else:
        st = st
    return(st)

In [488]:
def get_pair_string(pair, k):
    pre = []
    sur = []
    if pair[-1] != pair[0]:
        for item in pair:
            prefix_pair = item[:k-1]
            surfix_pair = item[k:]
            pre.append(prefix_pair)
            sur.append(surfix_pair)
    else:
        for item in pair:
            if item[:k-1] == item[k:]:
                ind = pair.index(item)
                break
        new_pair_1 = pair[ind:len(pair)]
        new_pair_2 = pair[1:ind+1]
        new_pair = new_pair_1 + new_pair_2
        for item in new_pair:
            prefix_pair = item[:k-1]
            surfix_pair = item[k:]
            pre.append(prefix_pair)
            sur.append(surfix_pair)
            
    return(pre, sur)

In [489]:
def get_genome_pair(pair, k, d):
    a = get_genome_string(get_pair_string(pair, k)[0], k-1)
    b = get_genome_string(get_pair_string(pair, k)[1], k-1)
    print('Prefix:', a)
    print('Surfix:', b)
    return(a[:k+d] + b)

We get the genome sequences of the above branch.

In [493]:
print('Genome sequences of the contig is:',get_genome_pair(vertexes, 4, 2))

Prefix: TGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGACTGCAACGGGCAATATGT
Surfix: TGTGTGGATTAAAAAAAGAGTGTCTGATAGCAACGGGCAATATGTCTCTGT
Genome sequences of the contig is: TGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAACGGGCAATATGTCTCTGT


With sequencing all maximal_branching_paths of the de Bruijn and thoroughly examing each branch, we will get the whole genome of Ecoli.